In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.0 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

In [3]:

import csv

def csv_to_list(file_path):
    result = []
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        #print(reader)
        idx = 0
        for row in reader:
            if(idx == 0):
                idx = True
                continue

            # print(row)
            description = row[0]
            labels = list(map(int, row[1:]))  # Convert the remaining columns to integers
            result.append([description, labels])
    return result

dataset = csv_to_list('/content/drive/My Drive/Re_Labeled_Dataset2.csv')
# print(dataset[0:10])
print(len(dataset))

104763


In [4]:
train_data = dataset[0 : 70000]
eval_data = dataset[70001 : 104763]


# Building and Training Model

In [5]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

model_args = MultiLabelClassificationArgs(num_train_epochs=1)
# Build model
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=41,
    args=model_args,
    attn_implementation="eager"
)

# Training
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)

# Make predictions with the model
# predictions, raw_outputs = model.predict(["Sam"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/69 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/348 [00:00<?, ?it/s]

In [6]:
print(result)

{'LRAP': 0.16009410187816053, 'eval_loss': 0.6938362693649599}


In [8]:
model_outputs[0]

array([0.50048828, 0.49536133, 0.51513672, 0.50390625, 0.55908203,
       0.48828125, 0.50585938, 0.46704102, 0.52832031, 0.47753906,
       0.52783203, 0.50488281, 0.49243164, 0.53564453, 0.48583984,
       0.48071289, 0.49389648, 0.55712891, 0.48876953, 0.46484375,
       0.49951172, 0.49511719, 0.49169922, 0.45849609, 0.50292969,
       0.48388672, 0.47192383, 0.53564453, 0.51855469, 0.53515625,
       0.47070312, 0.4987793 , 0.52685547, 0.4753418 , 0.48510742,
       0.53466797, 0.4921875 , 0.46923828, 0.46777344, 0.50390625,
       0.51171875])

In [9]:
categoryList = ['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE', 'COMEDY',
 'CRIME' ,'CULTURE & ARTS' ,'DIVORCE' ,'EDUCATION' ,'ENTERTAINMENT',
 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK' ,'GOOD NEWS' ,'GREEN', 'HEALTHY LIVING',
 'HOME & LIVING', 'IMPACT', 'LATINO VOICES', 'MEDIA' ,'MONEY', 'PARENTING',
 'PARENTS' ,'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE' ,'SPORTS', 'STYLE',
 'STYLE & BEAUTY', 'TASTE', 'TECH', 'TRAVEL', 'U.S. NEWS', 'WEDDINGS',
 'WEIRD NEWS', 'WELLNESS', 'WOMEN' ,'WORLD NEWS' ,'WORLDPOST']

In [31]:
# It's for confusion matrix
y_true = []
y_pred = []

In [32]:
import numpy as np
accuracy = 0
f1_score = 0
threshold = 0.5
numberOfRow = 0
for row in eval_data :
    # print(row)
    input = row[0]
    y_true.append(row[1])
    # print(input)
    # predictions, final_output = model.predict([input])
    # print(final_output)
    scores = model_outputs[numberOfRow]

    # print(scores)
    sorted_indices = np.argsort(scores)[::-1]
    # print(sorted_indices)
    pred = [categoryList[i] for i in sorted_indices if scores[i] > threshold]
    tmpList = []
    for i in model_outputs[numberOfRow] :
        if(i > threshold) :
            tmpList.append(1)
        else :
            tmpList.append(0)
    y_pred.append(tmpList)
    numberOfRow += 1
    trueList = []
    idx = 0
    # print(len(row[1]))
    for _ in row[1] :
        if(_ == 1):
            trueList.append(categoryList[idx])
        idx += 1
    trueList = set(trueList)

    accuracy += (len(trueList.intersection(pred)) / len(trueList.union(pred)))
    f1_score += (2 * len(trueList.intersection(pred))) / (len(trueList) + len(pred))


In [55]:
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Accuracy and f1-score

In [11]:
print(f"Accuracy : {accuracy / numberOfRow}")
print(f"f1-score : {f1_score / numberOfRow}")

Accuracy : 0.08780170071470182
f1-score : 0.15200734197155072


# Calculating the Confusion Matrix

In [57]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from google.colab import files

def plot_confusion_matrices(y_true, y_pred):
    num_labels = y_true.shape[1]
    for i in range(num_labels):
        cm = confusion_matrix(y_true[:, i], y_pred[:, i])
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
        disp.plot(cmap=plt.cm.Blues)
        plt.figure
        plt.title(f'Confusion Matrix for Label {categoryList[i]}')
        # plt.show()
        fileName = '/content/drive/My Drive/fig/' + f'Label_{categoryList[i]}(Baseline).png'
        # print(fileName)
        plt.savefig(fileName)
        plt.close()

plot_confusion_matrices(y_true, y_pred)